In [1]:
#!pip install -q langchain langchain-openai python-dotenv

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

if OPENAI_API_KEY.startswith("gsk_"):
    os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"
    MODEL_NAME = "llama-3.1-8b-instant"   

print("KEY HEAD:", OPENAI_API_KEY[:7])
print("MODEL   :", MODEL_NAME)

KEY HEAD: gsk_blU
MODEL   : llama-3.1-8b-instant


In [3]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai",   "키워드: {keywords}")
])

examples = [
    {
        "news": "삼성전자가 내년 초 자체 개발한 인공지능(AI) 가속기를 출시할 예정이다. 엔비디아 독점에 도전한다는 평가가 나온다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 새로운 건강 위기에 대응하기 위해 국제 협력 강화를 강조했다. 글로벌 보건 시스템 개선이 필요하다고 밝혔다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라가 자율주행 소프트웨어 업데이트를 발표하며 안전 기능을 대폭 개선했다고 전했다.",
        "keywords": "테슬라, 자율주행, 소프트웨어업데이트"
    },
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0.2,            
    api_key=OPENAI_API_KEY,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 뉴스 키워드 추출 전문가다. 다음 규칙을 반드시 지켜라.\n"
     "1) 핵심 키워드 정확히 3개만 추출\n"
     "2) '키워드: ' 다음에 쉼표(,)로 구분된 3개만 출력\n"
     "3) 불필요한 설명 금지, 요약문 금지, 해시태그 금지\n"
     "4) 가능한 한 고유명사/핵심개념 위주로 선택"),
    few_shot_prompt,
    ("human", "{input}")
])

parser = StrOutputParser()
chain = final_prompt | llm | parser


In [6]:
test_news = (
    "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 "
    "버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. "
    "모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) "
    "및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. "
    "내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
)

print(chain.invoke({"input": test_news}))


키워드: 제미나이, 구글 AI 스튜디오, 멀티모달


In [7]:
samples = [
    "국내 기준금리가 동결되면서 부동산 시장과 주식시장의 향방에 관심이 모이고 있다.",
    "맨체스터 시티가 프리미어리그 개막전에서 승리하며 연승 행진을 이어갔다.",
    "기후변화 대응을 위해 재생에너지 투자와 탄소중립 정책이 가속화되고 있다.",
]

for i, s in enumerate(samples, 1):
    print(f"\n[{i}] {s[:40]}...")
    print(chain.invoke({"input": s}))


[1] 국내 기준금리가 동결되면서 부동산 시장과 주식시장의 향방에 관심이 모이고...
키워드: 기준금리, 부동산시장, 주식시장

[2] 맨체스터 시티가 프리미어리그 개막전에서 승리하며 연승 행진을 이어갔다....
키워드: 맨체스터시티, 프리미어리그, 연승

[3] 기후변화 대응을 위해 재생에너지 투자와 탄소중립 정책이 가속화되고 있다....
키워드: 기후변화, 재생에너지, 탄소중립
